https://colab.research.google.com/drive/1e4O9gz-LS2vaZgZVgVwSxRpeZXQV_IHb

In [0]:
import numpy as np

In [0]:
docs = ["I enjoy playing TT", "I like playing TT"]

In [0]:
docs[0][0].split()

['I']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, token_pattern=r"\b\w+\b")
vectorizer.fit(docs)

print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(docs)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'i': 1, 'enjoy': 0, 'playing': 3, 'tt': 4, 'like': 2}
(2, 5)
<class 'scipy.sparse.csr.csr_matrix'>
[[1 1 0 1 1]
 [0 1 1 1 1]]


In [0]:
print(vectorizer.vocabulary_)
print(vector.shape)
print(vector.toarray())

{'i': 1, 'enjoy': 0, 'playing': 3, 'tt': 4, 'like': 2}
(2, 5)
[[1 1 0 1 1]
 [0 1 1 1 1]]


In [0]:
x = []
y = []
for i in range(len(docs)):
  for j in range(len(docs[i].split())):
    t_x = []
    t_y = []
    for k in range(4):
      if(j==k):
        t_y.append(docs[i].split()[k])
        continue
      else:
        t_x.append(docs[i].split()[k])
    x.append(t_x)
    y.append(t_y)

In [0]:
x

[['enjoy', 'playing', 'TT'],
 ['I', 'playing', 'TT'],
 ['I', 'enjoy', 'TT'],
 ['I', 'enjoy', 'playing'],
 ['like', 'playing', 'TT'],
 ['I', 'playing', 'TT'],
 ['I', 'like', 'TT'],
 ['I', 'like', 'playing']]

In [0]:
y

[['I'], ['enjoy'], ['playing'], ['TT'], ['I'], ['like'], ['playing'], ['TT']]

In [0]:
x2 = []
y2 = []
for i in range(len(x)):
  x2.append(' '.join(x[i]))
  y2.append(' '.join(y[i]))

In [0]:
x2

['enjoy playing TT',
 'I playing TT',
 'I enjoy TT',
 'I enjoy playing',
 'like playing TT',
 'I playing TT',
 'I like TT',
 'I like playing']

In [0]:
y2

['I', 'enjoy', 'playing', 'TT', 'I', 'like', 'playing', 'TT']

In [0]:
vector_x = vectorizer.transform(x2)
vector_x.toarray()

array([[1, 0, 0, 1, 1],
       [0, 1, 0, 1, 1],
       [1, 1, 0, 0, 1],
       [1, 1, 0, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1],
       [0, 1, 1, 0, 1],
       [0, 1, 1, 1, 0]])

In [0]:
vector_y = vectorizer.transform(y2)
vector_y.toarray()

array([[0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM , Bidirectional,Dropout
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
from keras import regularizers

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Dense(3, activation='linear', input_shape=(5,)))
model.add(Dense(5,activation='sigmoid'))
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 20        
Total params: 38
Trainable params: 38
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit(vector_x, vector_y, epochs=1000, batch_size=4,verbose=1)

In [0]:
model.predict(vector_x)

array([[0.13040131, 0.8821143 , 0.07797176, 0.08923113, 0.01265067],
       [0.153445  , 0.1215542 , 0.21058908, 0.36962688, 0.05544034],
       [0.11409801, 0.02018452, 0.22536606, 0.48198316, 0.06446832],
       [0.0532662 , 0.00465509, 0.01743671, 0.12412333, 0.8926726 ],
       [0.13284272, 0.9028559 , 0.07110441, 0.07974461, 0.0143581 ],
       [0.153445  , 0.1215542 , 0.21058908, 0.36962688, 0.05544034],
       [0.11627504, 0.02494821, 0.20845187, 0.45143813, 0.07265505],
       [0.05435377, 0.00577533, 0.01580954, 0.1113809 , 0.9043633 ]],
      dtype=float32)

In [0]:
[list(vectorizer.vocabulary_.keys())[0]]

['i']

In [0]:
vectorizer.transform([list(vectorizer.vocabulary_.keys())[1]]).toarray()

array([[1, 0, 0, 0, 0]])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 20        
Total params: 38
Trainable params: 38
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.models import Model
layer_name = 'dense_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

In [0]:
for i in range(len(vectorizer.vocabulary_)):
  word = list(vectorizer.vocabulary_.keys())[i]
  word_vec = vectorizer.transform([list(vectorizer.vocabulary_.keys())[i]]).toarray()
  print(word, '\t', intermediate_layer_model.predict(word_vec))

i 	 [[1.9760392 0.718137  1.5257659]]
enjoy 	 [[ 1.0395844 -0.9591169  0.5722229]]
playing 	 [[ 0.23517272 -1.3225136   0.24937609]]
tt 	 [[ 0.8973176  -0.08082079 -1.9826169 ]]
like 	 [[ 0.9075295 -1.0481944  0.5976531]]


# Measuring similarity between word vectors

In [0]:
a = word = list(vectorizer.vocabulary_.keys())[1]
word_vec_a = intermediate_layer_model.predict(vectorizer.transform([list(vectorizer.vocabulary_.keys())[1]]).toarray())

b = word = list(vectorizer.vocabulary_.keys())[4]
word_vec_b = intermediate_layer_model.predict(vectorizer.transform([list(vectorizer.vocabulary_.keys())[4]]).toarray())

In [0]:
word_vec_a

array([[ 1.0395844, -0.9591169,  0.5722229]], dtype=float32)

In [0]:
np.sum(word_vec_a*word_vec_b)/((np.sqrt(np.sum(np.square(word_vec_a))))*np.sqrt(np.sum(np.square(word_vec_b))))

0.99440795

In [0]:
np.sum(np.square(word_vec_a - word_vec_b))

0.026020005